In [5]:
!pip install pillow


  Using cached pillow-11.0.0-cp312-cp312-win_amd64.whl.metadata (9.3 kB)
Using cached pillow-11.0.0-cp312-cp312-win_amd64.whl (2.6 MB)


In [5]:
import os
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from collections import Counter, defaultdict
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import os
import pandas as pd
import cv2
from collections import Counter

# Load the dataset paths
base_path = "Indian-Traffic Sign-Dataset"
images_path = os.path.join(base_path, "Images")
labels_csv = os.path.join(base_path, "traffic_sign.csv")

# Load labels
labels_df = pd.read_csv(labels_csv)

# Count images in each class before merging
def count_images_in_class(path):
    class_counts = Counter()
    for class_id in os.listdir(path):
        class_dir = os.path.join(path, class_id)
        if os.path.isdir(class_dir):
            class_counts[class_id] = len(os.listdir(class_dir))
    return class_counts

print("Image counts before merging:")
class_counts_before = count_images_in_class(images_path)
for class_id, count in sorted(class_counts_before.items(), key=lambda x: int(x[0])):
    print(f"Class {class_id}: {count} images")

# Handle duplicates in the CSV
duplicate_groups = labels_df.groupby("Name")["ClassId"].apply(list)
merged_class_ids = {}
new_class_id = 0

for label, class_ids in duplicate_groups.items():
    for old_id in class_ids:
        merged_class_ids[old_id] = new_class_id
    new_class_id += 1

# Update CSV with merged class IDs
labels_df["ClassId"] = labels_df["ClassId"].map(merged_class_ids)
updated_csv_path = os.path.join(base_path, "traffic_sign_updated.csv")
labels_df.to_csv(updated_csv_path, index=False)
print("CSV file updated and saved to:", updated_csv_path)

# Merge image folders
new_images_path = os.path.join(base_path, "Images_Merged")
os.makedirs(new_images_path, exist_ok=True)

for old_class_id, new_class_id in merged_class_ids.items():
    old_class_dir = os.path.join(images_path, str(old_class_id))
    new_class_dir = os.path.join(new_images_path, str(new_class_id))
    os.makedirs(new_class_dir, exist_ok=True)
    
    if os.path.exists(old_class_dir):
        for img_file in os.listdir(old_class_dir):
            old_img_path = os.path.join(old_class_dir, img_file)
            new_img_path = os.path.join(new_class_dir, img_file)
            os.rename(old_img_path, new_img_path)

print("Image folders merged and saved to:", new_images_path)

# Count images in each class after merging
print("\nImage counts after merging:")
class_counts_after = count_images_in_class(new_images_path)
for class_id, count in sorted(class_counts_after.items(), key=lambda x: int(x[0])):
    print(f"Class {class_id}: {count} images")
